# Introdução

O algoritmo a seguir é do serviço de categorização da nossa plataforma, sendo um dos 4 serviços que temos (transcrição de aúdio em texto, criação de tags, categorização e resumo). Nele temos as fases de leitura de arquivos txt, tokenização, padronização em minúsculas, remoção de stop words, criação de dicionários com palavras chaves, contabilização de palavras-chaves e atribuição de categoria. Após a chamada da função main(), o retornado esperado é a categoria do vídeo em questão.

Para a Sprint 1 usaremos um trecho do podcast dos sócios ligados a tecnologia, chamado 'PROFISSÕES que serão SUBSTITUÍDAS por INTELIGÊNCIA ARTIFICAL | Os Sócios 120' e esperamos o retorno da categorização como 'Tecnologia'. A transcrição do vídeo foi feita pelo próprio youtube para uma maior assertividade.

Em outras sprints usaremos os aúdios transcritos dentro dos nossos serviços e com vídeos da Sambatech.

O link para o vídeo é o seguinte:

https://www.youtube.com/watch?v=Fen8aVyP32A


# Código

## Setup



In [ ]:
import nltk
from nltk import tokenize
import pandas as pd
import numpy
import re

## Definição de Funções


In [ ]:
def ler_arquivo(nome_arquivo):
    try:
        with open(nome_arquivo, 'r') as arquivo:
            linhas = arquivo.readlines()  # Lê todas as linhas do arquivo
            linhas_sem_linhas_em_branco = [linha.strip() for linha in linhas if linha.strip()]  # Remove linhas em branco
            conteudo = '\n'.join(linhas_sem_linhas_em_branco)  # Junta as linhas sem linhas em branco
        return conteudo
    except FileNotFoundError:
        print(f"O arquivo '{nome_arquivo}' não foi encontrado.")

nltk.download('punkt')
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('portuguese')


def tokenize_a_text(text):
  words = tokenize.word_tokenize(text, language='portuguese')
  return words

def remove_stop_words(words):
  stopwords = nltk.corpus.stopwords.words('portuguese')
  words_filtered = []
  for word in words:
    if word not in stopwords:
        words_filtered.append(word)
  return words_filtered

def lowercase_tokens(tokens):
    new_tokens = []
    for token in tokens:
        if isinstance(token, str):
            new_tokens.append(token.lower())
        else:  # Se for um "token" com múltiplas palavras
            print("token multiplo lower_case", token)
            new_words = [sub_word.lower() for sub_word in token]
            new_tokens.extend(new_words)
    return new_tokens


def pipeline_pre_processing (text):
  tokenize_text = tokenize_a_text(text)
  lowercase_text = lowercase_tokens(tokenize_text)
  filtered_text = remove_stop_words(lowercase_text)

  return filtered_text

def contabiliza_palavras(df, coluna):
    for indice, linha in df.iterrows():
        texto = linha[coluna]
        score_tecnologia = 0
        score_empreendedorismo = 0
        score_vendas = 0
        score_lideranca = 0
        score_estrategia = 0

        for palavra in texto:
            if palavra in palavras_tecnologia:
                score_tecnologia += 1
            if palavra in palavras_empreendedorismo:
                score_empreendedorismo += 1
            if palavra in palavras_vendas:
                score_vendas += 1
            if palavra in palavras_lideranca:
                score_lideranca += 1
            if palavra in palavras_estrategia:
                score_estrategia += 1

        df.at[indice, 'score_tecnologia'] = score_tecnologia
        df.at[indice, 'score_empreendedorismo'] = score_empreendedorismo
        df.at[indice, 'score_vendas'] = score_vendas
        df.at[indice, 'score_lideranca'] = score_lideranca
        df.at[indice, 'score_estrategia'] = score_estrategia

    return df

def atribui_categoria(df):
  for indice, linha in df.iterrows():
      categorias = ['tecnologia', 'empreendedorismo', 'vendas', 'lideranca', 'estrategia']
      scores = [linha['score_tecnologia'], linha['score_empreendedorismo'], linha['score_vendas'],
                linha['score_lideranca'], linha['score_estrategia']]
      maior_score = max(scores)
      indice_maior_score = scores.index(maior_score)
      categoria = categorias[indice_maior_score]
      df.at[indice, 'category'] = categoria
  return df

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Testes da Função


## 1.1 Leitura de Arquivos .txt

In [ ]:
expected_outcome ='Esse texto foi transcrito'
nome_do_arquivo = '/content/sample_data/testetxt.txt'  # Nome do arquivo que você quer ler
texto = ler_arquivo(nome_do_arquivo)
print(texto == expected_outcome)

True


## 1.2 Tokenização

In [ ]:
text = "O Inteli é excelente"

words = tokenize_a_text(text)

expected_outcome = ['O', 'Inteli', 'é', 'excelente']

print(words == expected_outcome)

True


## 1.3 Remoção Stop Words

In [ ]:
tokens = ['O', 'Inteli', 'é', 'excelente']

text_filtered = remove_stop_words(tokens)

expected_outcome = ['O', 'Inteli', 'excelente']

print(text_filtered == expected_outcome)

True


## 1.4 Padronização em Minúsculas

In [ ]:
tokens = ['O', 'Inteli', 'é', 'excelente']

lower_text = lowercase_tokens(tokens)

expected_outcome = ['o', 'inteli', 'é', 'excelente']

print(lower_text == expected_outcome)

True


## 1.5 Contabiliza Palavras

In [ ]:
# O category como false não significa que o teste não teve sucesso por que ele é agnóstico a função e deve retornar false

filtered_text_test = ["liderança",    "equipe",    "motivação",    "gestão",    "inovação",    "cultura",    "empreendedorismo",    "start-up",    "investimento",    "modelo de negócio",    "marketing",    "crescimento",    "sustentabilidade",    "software",    "hardware",    "inteligência artificial",    "machine learning",    "blockchain",    "realidade virtual",    "robótica",    "automação",    "dados",    "cybersegurança",    "cloud computing",    "iot",    "internet das coisas",    "aplicativo",    "tecnologia",    "digital"]

df = pd.DataFrame(columns=['filtered_text','score_tecnologia','score_empreendedorismo','score_estrategia','score_vendas','score_lideranca','category'])
df.loc[0,'filtered_text'] = filtered_text_test

df_expected_outcome = df
df_expected_outcome.loc[0, ['score_tecnologia','score_empreendedorismo','score_estrategia','score_vendas','score_lideranca']] = [17,2,0,0,5]


df_response = df
df_response = contabiliza_palavras(df_response, 'filtered_text')
print(df_response == df_expected_outcome)

   filtered_text  score_tecnologia  score_empreendedorismo  score_estrategia  \
0           True              True                    True              True   

   score_vendas  score_lideranca  category  
0          True             True     False  


## 1.6 Atribui Categoria

In [ ]:
df_expected_outcome = df
df_expected_outcome.loc[0, ['score_tecnologia','score_empreendedorismo','score_estrategia','score_vendas','score_lideranca','category']] = [17,2,0,0,5,"Tecnologia"]

df_response = df
df_response = atribui_categoria(df_response)

print(df_response == df_expected_outcome)

   filtered_text  score_tecnologia  score_empreendedorismo  score_estrategia  \
0           True              True                    True              True   

   score_vendas  score_lideranca  category  
0          True             True      True  


# Main

In [ ]:
# Para fins de teste, é necessário adicionar o arquivo txt no sample_data que esta no respositório do Git
def main():

  nome_do_arquivo = '/content/sample_data/OSSOCIOS.txt'
  df = pd.DataFrame(columns=['transcription','filtered_text','score_tecnologia','score_empreededorismo','score_estrategia','score_vendas','score_lideranca','category'])
  palavras_tecnologia =[ "tecnologia",   "inovação",    "software",    "hardware",    "inteligência artificial",    "machine learning",    "blockchain",    "realidade virtual",    "realidade aumentada",    "robótica",    "automação",    "dados",    "algoritmo",    "cybersegurança",    "rede",    "cloud computing",    "iot",    "internet das coisas",    "desenvolvimento",    "aplicativo",    "sistema operacional banco de dados",    "programação",    "interface",    "usabilidade",    "tecnologia",    "5g",    "digital"]
  palavras_empreendedorismo = [    "empreendedorismo",    "start-up",    "escalabilidade",    "investimento",    "venture",    "capital",    "pitch",    "modelo de negócio",    "empreendedor",    "inovação",    "mercado",    "produto",    "serviço",    "cliente",    "parceria",    "lucro",    "receita",    "competição",    "branding",    "marketing",    "crescimento",    "sustentabilidade",    "empreender",    "networking",    "incubadora",    "aceleradora",    "mvp"]
  palavras_vendas = ["vendas",    "negociação",    "conversão",    "cliente",    "produto",    "serviço",    "estratégia de vendas",    "funil de vendas",    "marketing digital",    "lead",    "crm",    "proposta",    "fechamento",    "objeção",    "desconto",    "meta",    "comissão",    "vendedor",    "prospeção",    "engajamento",    "retenção",    "pós-venda",    "fidelização",    "upsell",    "cross-sell",    "roi"]
  palavras_lideranca = [    "liderança",    "líder",    "equipe",    "motivação",    "gestão",    "comunicação",    "visão",    "objetivo",    "feedback",    "desenvolvimento",    "empoderamento",    "estratégia",    "conflito",    "inovação",    "cultura",    "comprometimento",    "resiliência",    "coaching",    "liderança servidora",    "diversidade", "inclusão",    "tomada de decisão",    "negociação",    "inteligência emocional",    "autoridade",    "inspiração"]
  palavras_estrategia = [    "estratégia",    "planejamento",    "objetivo",    "meta",    "análise",    "swot (forças, fraquezas, oportunidades, ameaças)",    "competitividade",    "mercado",    "inovação",    "crescimento",    "escala",    "diferenciação",    "posicionamento",    "execução",    "kpi (indicadores-chave de performance)",    "benchmarking",    "vantagem competitiva",    "modelo de negócio",    "análise de risco",    "gestão de mudanças",    "sustentabilidade",    "segmentação",    "alvo",    "tática",    "otimização",    "visão"]


  texto = ler_arquivo(nome_do_arquivo)
  df.loc[0, 'transcription'] = texto
  df.head()
  filtered_text = pipeline_pre_processing(texto)
  df['filtered_text'] = df['filtered_text'].apply(lambda x: filtered_text if pd.isna(x) else x)
  df = contabiliza_palavras(df, 'filtered_text')
  df = atribui_categoria(df)
  df.head()
  categoria = df.loc[0,'category']
  print('Categoria:', categoria)

  return categoria

categoria = main()




Categoria: tecnologia
